In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time
from tqdm.notebook import tqdm

In [2]:
def extract_features(listing):
    try:
        bedrooms_div = re.sub(r"[\n\t\s]*", "", listing.find("span", title="Bedrooms").text)
    except:
        bedrooms_div = None

    try:
        bathrooms_div = re.sub(r"[\n\t\s]*", "", listing.find("span", title="Bathrooms").text)
    except:
        bathrooms_div = None

    try:
        parking_div = re.sub(r"[\n\t\s]*", "", listing.find("span", title="Parking Spaces").text)
    except:
        parking_div = None

    try:
        floor_size_div = re.sub(r"[\n\t\s]*", "", listing.find("span", title="Floor Size").text)[:-2]
    except:
        floor_size_div = None

    try:
        erf_size_div = re.sub(r"[\n\t\s]*", "", listing.find("span", title="Erf Size").text)[:-2]
    except:
        erf_size_div = None
            
    return [bedrooms_div, bathrooms_div, parking_div, floor_size_div, erf_size_div]

In [3]:
def scrape_property_data(property_24_url):
    listings_per_page = []
    
    property_24_page = BeautifulSoup(property_24_url, 'html.parser')
    listings_divs = property_24_page.find_all("div", class_= "p24_content")
    listings_spans = property_24_page.find_all("span", class_= "p24_content")
    
    for listing in listings_divs:
        try:
            price_div = re.sub(r"[\n\t\s]*", "", listing.find("div", class_="p24_price").text).replace("R", "")            
        except:
            price_div = None

#         print([price_div] + extract_features(listing))
        listings_per_page.append([price_div] + extract_features(listing))
        
    for listing in listings_spans:
        try:
            price_div = re.sub(r"[\n\t\s]*", "", listing.find("span", class_="p24_price").text).replace("R", "")              
        except:
            price_div = None

#         print([price_div] + extract_features(listing))
        listings_per_page.append([price_div] + extract_features(listing))
        
    return listings_per_page

In [4]:
listings_with_developments_and_poa = [["Price", "Bedrooms", "Bathrooms", "Parking", "Floor Size", "Erf Size"]]
developments_and_poa = []

for page_number in tqdm(range(1,441)):
    
    if page_number == 1: 
        property_24_url = requests.get("https://www.property24.com/for-sale/cape-town/western-cape/432").text
    else:
        property_24_url = requests.get("https://www.property24.com/for-sale/cape-town/western-cape/432/p{}".format(page_number)).text
        time.sleep(10)
        
        
    listings_per_page = scrape_property_data(property_24_url)
    for listing in listings_per_page:
        listings_with_developments_and_poa.append(listing)
    
    for listing in listings_per_page:
        for features in listing:
            if "From" in str(features):
                developments_and_poa.append(listing)
            elif "POA" in str(features):
                developments_and_poa.append(listing)
    

In [16]:
listings = [listing for listing in listings_with_developments_and_poa if listing not in developments_and_poa]
listings_df = pd.DataFrame(listings)
listings_df.columns = listings_df.iloc[0]
listings_df = listings_df[1:]
# listings_df.to_csv(r"C:\Users\kile\OneDrive - Esri South Africa\DSI Programme\Module 1\Twist Challenge\Data\listings.csv", index=False)
print("Listings:\n")
display(listings_df)

Listings:



,Price,Bedrooms,Bathrooms,Parking,Floor Size,Erf Size
1,1265000,1,1,1,61,None
2,4950000,None,None,None,None,2448
3,1385000,1,1,1,51,None
4,1385000,1,1,1,51,None
5,9900000,6,3,8,None,677
...,...,...,...,...,...,...
9097,21000000,None,None,None,1450,None
9098,790000,2,1,1,56,None
9099,3750000,None,None,None,None,8000
9100,18500000,None,None,None,2941,None
